# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# pip install gmaps
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
#api_keys = "../api_keys.py"
#api_keys = "C:\\Users\\joeki\\Desktop\\python-api-challenge\\api_keys.py"
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../output_data/cities.csv")
weather_data.head()

,City ID,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,upernavik,72.7868,-56.1549,18.70,76,90,2.15,GL,1612657442
1,1,butaritari,3.0707,172.7902,81.21,80,92,20.56,KI,1612657726
2,2,puerto baquerizo moreno,-0.9000,-89.6000,77.13,82,31,8.41,EC,1612657726
3,3,punta arenas,-53.1500,-70.9167,53.60,62,0,18.41,CL,1612657727
4,4,yellowknife,62.4560,-114.3525,-36.40,67,1,2.30,CA,1612657691


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Configure gmaps
gmaps.configure(api_key=g_key)

locations = weather_data[["Lat", "lng"]].astype(float)

humidity = weather_data["Humidity"].astype(float)

In [ ]:
# Plot Heatmap
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)
plt.savefig("C:/Users/joeki/Desktop/python-api-challenge/output_data/heatmap.png")

# Display figure
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
narrow_city_df = weather_data.loc[(weather_data["Max Temp"] <=80) & (weather_data["Max Temp"] >=70) &
    (weather_data["Wind Speed"] <=10) & (weather_data["Cloudiness"] ==0)].dropna()
narrow_city_df                             

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = narrow_city_df.loc[:,["City","Country", "Lat", "lng", "Max Temp"]]

hotel_df["Hotel Name"] = ""

hotel_df


In [ ]:
endpoint_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [ ]:

for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from hotel_df
    lat = row["Lat"]
    lng = row["lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Getting the Results for Index {index}: '{city_name}'.")
    response = requests.get(endpoint_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in '{city_name}' is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    #time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

In [ ]:
selected_df = hotel_df.head(10)
selected_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in selected_df.iterrows()]
locations = selected_df[["Lat", "lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig
